In [1]:
import numpy as np
import pandas as pd
import matlab.engine
import yaml
import time
import nolitsa
import importlib
import pathlib
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
from scipy.stats import pearsonr
from loguru import logger
from Entropy import ApproximateEntropy
from functools import partial, wraps
from itertools import product
import os
from typing import Union
from numbers import Number
from utilities import ZScore

In [12]:
import importlib
import Nonlinear
importlib.reload(Nonlinear)

<module 'Nonlinear' from '/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Operations/Nonlinear.py'>

In [3]:
def compare_outputs(outputs, exclude_keys=None):
    """
    Compare MATLAB and Python feature outputs, computing relative errors
    and Pearson correlations.

    Parameters
    ----------
    outputs : dict
        Nested dictionary of feature values with structure:
        {
            feature_name: {
                ts_id: {
                    'matlab': scalar_or_dict,
                    'python': scalar_or_dict
                }
            }
        }
    exclude_keys : set or list, optional
        Set or list of keys (feature names or feature.subkey) to exclude from comparison.

    Returns
    -------
    results : dict
        Dictionary mapping feature (or subfeature) names to correlation and stats:
        {
            'feature.subkey': {
                'r': float,
                'pval': float,
                'res_py': ndarray,
                'res_matlab': ndarray,
                'max_rel_err': float
            }
        }
    """
    if exclude_keys is None:
        exclude_keys = set()
    else:
        exclude_keys = set(exclude_keys)

    flat = {}
    for feat, ts_dict in outputs.items():
        for ts, run in ts_dict.items():
            ml = run['matlab']
            py = run['python']
            if isinstance(ml, dict) and isinstance(py, dict):
                for k, mlv in ml.items():
                    slot = f"{feat}.{k}"
                    if slot in exclude_keys or k not in py:
                        continue
                    pyv = py[k]
                    flat.setdefault(slot, {})[ts] = (mlv, pyv)
            elif isinstance(ml, Number) and isinstance(py, Number):
                if feat in exclude_keys:
                    continue
                flat.setdefault(feat, {})[ts] = (ml, py)
            else:
                raise ValueError(f"Feature {feat}@{ts} is neither both scalars nor both dicts.")

    results = {}
    for slot, tsmap in flat.items():
        ml_vals, py_vals = [], []
        rel_errors = []

        for ts, (mlv, pyv) in tsmap.items():
            ml_vals.append(mlv)
            py_vals.append(pyv)

            both_finite = np.isfinite(mlv) and np.isfinite(pyv)
            both_nan = np.isnan(mlv) and np.isnan(pyv)
            both_posinf = (mlv == np.inf) and (pyv == np.inf)
            both_neginf = (mlv == -np.inf) and (pyv == -np.inf)

            if both_finite:
                if mlv == 0:
                    rel_err = np.nan
                else:
                    rel_err = abs(mlv - pyv) / abs(mlv) * 100
                    rel_errors.append(rel_err)
                print(f"[{slot} | ts={ts}]  RelErr% = {rel_err:.2f}")
            elif both_nan or both_posinf or both_neginf:
                print(f"[{slot} | ts={ts}]  RelErr% = MATCH (both non-finite)")
            else:
                print(f"[{slot} | ts={ts}]  RelErr% = NaN (mismatch in finiteness)")

        ml_arr = np.array(ml_vals, dtype=float)
        py_arr = np.array(py_vals, dtype=float)
        finite_mask = np.isfinite(ml_arr) & np.isfinite(py_arr)

        if finite_mask.sum() > 1 and ml_arr[finite_mask].std() and py_arr[finite_mask].std():
            r, p = pearsonr(ml_arr[finite_mask], py_arr[finite_mask])
        else:
            r, p = np.nan, np.nan

        max_rel_err = np.nanmax(rel_errors) if rel_errors else np.nan

        results[slot] = {
            'r': r,
            'pval': p,
            'res_py': py_arr,
            'res_matlab': ml_arr,
            'max_rel_err': max_rel_err
        }

    return results

In [4]:
empirical1000 = []
with open("../../../empirical1000/hctsa_timeseries-data.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        # Convert each element from string to float (or int if appropriate)
        try:
            time_series = [float(value) for value in row if value != '']
            empirical1000.append(time_series)
        except ValueError as e:
            print(f"Skipping row due to conversion error: {row}")
            continue

In [5]:
def load_yaml(file):
    print(f"Loading configuration file: {file.split('/')[-1]}")
    funcs = {}
    with open(file) as f:
        yf = yaml.load(f, Loader=yaml.FullLoader)

    for module_name in yf:
        print(f"\n*** Importing module {module_name} *** \n")
        module = importlib.import_module(module_name)
        for function_name in yf[module_name]:
            # Get the function's configuration dictionary
            function_config = yf[module_name][function_name]
            # If no configs section exists or if it's empty, use a list with single empty dict
            if ('configs' not in function_config or function_config.get('configs') is None or 
                function_config.get('configs') == []):
                configs = [{}]
            else:
                configs = function_config.get('configs', [{}])

            for params in configs:
                # Handle the case where params is None
                if params is None:
                    params = {}
                    
                zscore_first = params.pop("zscore", False)
                param_keys, param_vals = zip(*params.items()) if params else ([], [])
                
                param_combinations = [dict(zip(param_keys, values)) 
                                   for values in product(*[v if isinstance(v, list) 
                                                        else [v] for v in param_vals])]
                
                # If no parameter combinations were generated, add empty dict
                if not param_combinations:
                    param_combinations = [{}]
                
                # create a function for each parameter combination
                for param_set in param_combinations:
                    feature_name = (f"{module_name}_{function_name}_" + 
                                  "_".join(f"{v}" for k, v in param_set.items())
                                  if param_set else f"{module_name}_{function_name}")
                    if not zscore_first:
                        feature_name += "_raw"
                    
                    print(f"Adding operation {feature_name} with params {param_set} "
                          f"(Z-score={zscore_first})")
                    
                    base_func = partial(getattr(module, function_name), **param_set)
                    if zscore_first:
                        base_func = zscore_decorator(base_func)
                    
                    # return the MATLAB callable corresponding to the python implementation for direct comparison
                    # make sure to check whethe the data needs to be zscored when calling the MATLAB func, cannot be wrapped as it is not a python function
                    # so needs to be done manually when calling the function.
                    hctsa_name = function_config.get('hctsa_name')
                    hctsa_callable = eval(f"eng.{hctsa_name}")

                    # keep ordered args only for testing YAML otherwise bloats
                    funcs[feature_name] = {'callable': base_func, 'params': param_set, 'hctsa_name': function_config.get('hctsa_name'), 
                                           'matlab_callable': hctsa_callable, 'isZscore': zscore_first, 'ordered_args': function_config.get('ordered_args')}
                    
    return funcs

In [ ]:
def eval_comparison(yaml, data):
    func_dict = load_yaml(yaml)
    func_res = dict()
    for func in func_dict:
        print(f"Evalutating {func}")
        f = func_dict[func]
        python_func = f['callable']
        matlab_func = f['matlab_callable']
        hctsa_name = f['hctsa_name']
        isZscore = f['isZscore']
        params = f['params']
        ordered_args = []
        if params:
            order = f['ordered_args']
            ordered_args = [params[k] for k in order]
            
        print(f"Comparing to {hctsa_name}")
        res = dict()
        for i in range(len(data)):
            x = np.array(data[i]).reshape(-1, 1)
            matlab_eval = matlab_func(ZScore(matlab.double(x)), *ordered_args) if isZscore else matlab_func(matlab.double(x), matlab.double(ordered_args[0]))
            python_eval = python_func(x)
            res[i] = {'matlab': matlab_eval, 'python': python_eval}
        func_res[func] = res
    return func_res

In [7]:
eng = matlab.engine.start_matlab()

In [8]:
proj_root = pathlib.Path("/Users/jmoo2880/Documents/hctsa")
eng.addpath(eng.genpath(str(proj_root)), nargout=0)
def zscore_decorator(func):
    @wraps(func)
    def wrapper(y, *args, **kwargs):
        y = ZScore(y)
        return func(y, *args, **kwargs)
    return wrapper

def range_constructor(loader, node):
    start, end = loader.construct_sequence(node)
    return list(range(start, end+1))
yaml.add_constructor("!range", range_constructor)

In [35]:
funcs = load_yaml("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/nonlinear.yaml")

Loading configuration file: nonlinear.yaml

*** Importing module Nonlinear *** 

Adding operation Nonlinear_FNN_[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]_1_5.0_1 with params {'de': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'tau': 1, 'th': 5.0, 'kth': 1} (Z-score=True)


In [18]:
fdfa_res = eval_comparison("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/nonlinear.yaml", empirical1000[1:5])

Loading configuration file: nonlinear.yaml

*** Importing module Nonlinear *** 

Adding operation Nonlinear_FNN_5.0_1 with params {'th': 5.0, 'kth': 1} (Z-score=True)
Evalutating Nonlinear_FNN_5.0_1
Comparing to NL_MS_fnn


In [25]:
eng.NL_MS_fnn(ZScore(matlab.double(empirical1000[0])), matlab.double(list(range(1, 11))), 1, 5, 1)

{'pfnn_1': 0.9994345170775842,
 'pfnn_2': 0.9356871374274855,
 'pfnn_3': 0.7167150145043513,
 'pfnn_4': 0.4610844337735094,
 'pfnn_5': 0.2337168584292146,
 'pfnn_6': 0.10036021612967781,
 'pfnn_7': 0.039727809466626636,
 'pfnn_8': 0.009807846277021618,
 'pfnn_9': 0.0028025222700430387,
 'pfnn_10': 0.0004004004004004004,
 'meanpfnn': 0.3499736755755915,
 'stdpfnn': 0.40030843028836943,
 'firstunder02': 6.0,
 'firstunder01': 7.0,
 'firstunder005': 7.0,
 'firstunder002': 8.0,
 'firstunder001': 8.0,
 'max1stepchange': 0.2556305807308419}

In [36]:
funcs['Nonlinear_FNN_[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]_1_5.0_1']['callable'](empirical1000[0])

{'pfnn_1': np.float64(0.9995476136620675),
 'pfnn_2': np.float64(0.9356871374274855),
 'pfnn_3': np.float64(0.7167150145043513),
 'pfnn_4': np.float64(0.4610844337735094),
 'pfnn_5': np.float64(0.2337168584292146),
 'pfnn_6': np.float64(0.10036021612967781),
 'pfnn_7': np.float64(0.039727809466626636),
 'pfnn_8': np.float64(0.009807846277021618),
 'pfnn_9': np.float64(0.0028025222700430387),
 'pfnn_10': np.float64(0.0004004004004004004),
 'pfnn_11': np.float64(0.0004004404845329863),
 'meanpfnn': np.float64(0.3182045720749937),
 'stdpfnn': np.float64(0.39414056908907785),
 'firstunder02': 6,
 'firstunder01': 7,
 'firstunder005': 7,
 'firstunder002': 8,
 'firstunder001': 8,
 'max1stepchange': np.float64(0.2556305807308419)}